In [1]:
import numpy as np

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from structure.Dataset import Dataset
from structure.SklearnEnsemble import SklearnEnsemble
from predict import majority_voting

/Users/tomek/.virtualenvs/masters/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Users/tomek/.virtualenvs/masters/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
iris = load_iris()
train_data, val_data = Dataset(iris.data, iris.target, iris.feature_names, iris.target_names).split()

tree = DecisionTreeClassifier(max_depth=2)
clf = AdaBoostClassifier(n_estimators=10, base_estimator=tree)

ensemble = SklearnEnsemble(clf)
ensemble.fit(train_data)

In [16]:
preds = ensemble.predict(val_data.X)
preds = np.rollaxis(preds, axis=1)
print(preds)
print(preds.shape)
majority_voting(preds), val_data.y

[[2 2 2 2 2 2 2 1 1 2]
 [2 2 2 2 2 2 2 2 2 2]
 [0 2 0 2 0 2 0 1 1 0]
 [0 2 0 2 0 2 0 1 1 0]
 [2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 1 1 2]
 [2 2 2 2 2 2 2 1 1 2]
 [2 2 2 2 2 2 2 1 1 2]
 [0 2 0 0 0 2 0 1 1 0]
 [0 2 0 0 0 2 0 1 1 0]
 [2 2 2 2 2 2 2 1 1 2]
 [2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2]]
(15, 10)


(array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2]),
 array([1, 2, 0, 0, 2, 2, 1, 1, 1, 0, 0, 1, 2, 1, 2]))